In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_excel('/content/final_data.xlsx')

In [ ]:
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,sentiment,sentiment_score
0,2539,clean & quiet apt home by the park,2787,john,brooklyn,kensington,40.64749,-73.97237,private room,149,1,9,2018-10-19,0.21,6,365,positive,0.516888
1,2595,skylit midtown castle,2845,jennifer,manhattan,midtown,40.75362,-73.98377,entire home/apt,225,1,45,2019-05-21,0.38,2,355,neutral,0.747688
2,3647,the village of harlem....new york !,4632,elisabeth,manhattan,harlem,40.80902,-73.94190,private room,150,3,0,2019-05-21,0.72,1,365,neutral,0.820897
3,3831,cozy entire floor of brownstone,4869,lisaroxanne,brooklyn,clinton hill,40.68514,-73.95976,entire home/apt,89,1,270,2019-07-05,4.64,1,194,neutral,0.546140
4,5022,entire apt: spacious studio/loft by central park,7192,laura,manhattan,east harlem,40.79851,-73.94399,entire home/apt,80,10,9,2018-11-19,0.10,1,0,neutral,0.631164


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48884 entries, 0 to 48883
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   id                              48884 non-null  int64         
 1   name                            48865 non-null  object        
 2   host_id                         48884 non-null  int64         
 3   host_name                       48857 non-null  object        
 4   neighbourhood_group             48884 non-null  object        
 5   neighbourhood                   48884 non-null  object        
 6   latitude                        48884 non-null  float64       
 7   longitude                       48884 non-null  float64       
 8   room_type                       48884 non-null  object        
 9   price                           48884 non-null  int64         
 10  minimum_nights                  48884 non-null  int64         
 11  nu

# Checking assumptions for ANOVA

In [ ]:
import pandas as pd
from scipy.stats import f_oneway

# Assuming your dataframe is df

# Extract price values for each sentiment group
positive_prices = df[df['sentiment'] == 'positive']['price']
neutral_prices = df[df['sentiment'] == 'neutral']['price']
negative_prices = df[df['sentiment'] == 'negative']['price']

from scipy.stats import shapiro

print(shapiro(positive_prices.sample(500)))  # sample to avoid crash
print(shapiro(neutral_prices.sample(500)))
print(shapiro(negative_prices))

# 2. Homogeneity of variances (Levene’s Test)
from scipy.stats import levene

stat, p = levene(positive_prices, neutral_prices, negative_prices)
print("Levene’s p-value:", p)



ShapiroResult(statistic=np.float64(0.5960197859368426), pvalue=np.float64(1.7871291015735768e-32))
ShapiroResult(statistic=np.float64(0.13459302534769813), pvalue=np.float64(1.1780887300916203e-42))
ShapiroResult(statistic=np.float64(0.7662109011561405), pvalue=np.float64(2.795388061129556e-13))
Levene’s p-value: 3.787029595414033e-09


# Kruskal Wallis Test (assumptions failed)

In [ ]:
from scipy.stats import kruskal

# Kruskal-Wallis Test
stat, p_value = kruskal(positive_prices, neutral_prices, negative_prices)

print("Kruskal-Wallis H-Test Results")
print("Statistic:", stat)
print("p-value:", p_value)

alpha = 0.05
if p_value < alpha:
    print("\nReject Null Hypothesis: At least one sentiment group has a significantly different average price.")
else:
    print("\nFail to Reject Null Hypothesis: No significant difference in average price across sentiment groups.")


Kruskal-Wallis H-Test Results
Statistic: 31.913741990002148
p-value: 1.1749488984355748e-07

✅ Reject Null Hypothesis: At least one sentiment group has a significantly different average price.


In [ ]:
%pip install scikit-posthocs

In [ ]:
import pandas as pd
import scikit_posthocs as sp

# Prepare data in required format
data = df[['sentiment', 'price']]

# Run Dunn's post-hoc test
dunn_result = sp.posthoc_dunn(data, val_col='price', group_col='sentiment', p_adjust='bonferroni')

print("\nDunn Post-Hoc Test (with Bonferroni Correction):")
print(dunn_result)


Dunn Post-Hoc Test (with Bonferroni Correction):
          negative   neutral  positive
negative  1.000000  0.003526  0.000610
neutral   0.003526  1.000000  0.000023
positive  0.000610  0.000023  1.000000
